<a href="https://colab.research.google.com/github/BalytskyiJaroslaw/PathogensRamanOpenSet/blob/main/EOS_K_p1_I_p2_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Operating in the ***open*** world, Entropic Open Set, $\mathcal{K} = p_1$, $\mathcal{I} = p_2$

In [ ]:
from __future__ import absolute_import
from __future__ import print_function

import numpy as np
from numpy import genfromtxt
from sklearn.utils import shuffle


import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from pandas import read_csv

from keras.layers import Lambda, Multiply

import csv
import pprint

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

import keras
#from keras.utils import to_categorical
from pandas import read_csv
from keras.models import Sequential
from keras import layers
#from keras.optimizers import SGD
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense
#from keras.optimizers import SGD

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import random
from pprint import pprint

from sklearn.preprocessing import LabelEncoder
import gspread
import pandas as pd
import pickle
import pywt
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
#from keras.utils import to_categorical

from time import time
t00 = time()
import os
from tensorflow.keras import layers
from keras.layers import Input, Dense, concatenate
from keras.models import Model
from keras.layers import GlobalAveragePooling1D

In [ ]:
# Initialize the TPU and spread the computations across the 8 cores
import tensorflow as tf
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
tf.config.list_logical_devices('TPU')
#print("All devices: ", tf.config.list_logical_devices('TPU'))

strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


# Reference dataset

In [ ]:
# Reference data
data_X_reference = np.load("/content/gdrive/MyDrive/Stanford_data/X_reference.npy")
# Test data
data_X_test = np.load("/content/gdrive/MyDrive/Stanford_data/X_test.npy")

data_y_reference = np.load("/content/gdrive/MyDrive/Stanford_data/y_reference.npy")
# Test labels
data_y_test = np.load("/content/gdrive/MyDrive/Stanford_data/y_test.npy")

data_y_reference_int = []

for i in range(data_y_reference.shape[0]):
  data_y_reference_int.append(int(data_y_reference[i]))

data_y_test_int = []

for i in range(data_y_test.shape[0]):
  data_y_test_int.append(int(data_y_test[i]))

train_label = tf.keras.utils.to_categorical(data_y_reference_int)
test_label = tf.keras.utils.to_categorical(data_y_test_int)

# $\mathcal{K} = p_1$, reference indices, training dataset: 18000 - 26000, 34000 - 38000, 44000 - 50000, 56000 - 58000. $\mathcal{I} = p_2$, reference indices: 28000 - 34000, 52000 - 56000.

In [ ]:
indices =[slice(18000, 26000),slice(34000, 38000),slice(44000, 50000),slice(56000, 58000),slice(28000, 34000),slice(52000, 56000)]

Known_and_Ignored_data_X_reference = np.concatenate([data_X_reference[idx, :] for idx in indices], axis=0)

Known_and_Ignored_data_X_train_label_int = []

for i in range(2000*10):
  Known_and_Ignored_data_X_train_label_int.append(int(data_y_reference[i]))

for i in range(2000*5):
 Known_and_Ignored_data_X_train_label_int.append(int(data_y_reference[2000*10]))

Known_and_Ignored_data_X_train_label = tf.keras.utils.to_categorical(Known_and_Ignored_data_X_train_label_int)
print("Shape of the known and ignored data, reference:", Known_and_Ignored_data_X_reference.shape)
print("Shape of the known and ignored labels, reference:", Known_and_Ignored_data_X_train_label.shape)

Shape of the known and ignored data, reference: (30000, 1000)
Shape of the known and ignored labels, reference: (30000, 11)


# $\mathcal{K}$, known reference indices for the ***test***: 900 - 1300, 1700 - 1900, 2200 - 2500, 2800 - 2900. Ignored, ***test***: 1400 - 1700, 2600 - 2800.

In [ ]:
indices =[slice(900, 1300),slice(1700, 1900),slice(2200, 2500),slice(2800, 2900),slice(1400, 1700),slice(2600, 2800)]

Known_and_Ignored_data_X_test = np.concatenate([data_X_test[idx, :] for idx in indices], axis=0)

Known_and_Ignored_data_X_test_label_int = []

for i in range(100*10):
  Known_and_Ignored_data_X_test_label_int.append(int(data_y_test[i]))

for i in range(100*5):
  Known_and_Ignored_data_X_test_label_int.append(int(data_y_test[100*10]))

Known_and_Ignored_data_X_test_label = tf.keras.utils.to_categorical(Known_and_Ignored_data_X_test_label_int)
print("Shape of the known and ignored data, reference:", Known_and_Ignored_data_X_test.shape)
print("Shape of the known and ignored labels, reference:", Known_and_Ignored_data_X_test_label.shape)

Shape of the known and ignored data, reference: (1500, 1000)
Shape of the known and ignored labels, reference: (1500, 11)


# Leave only $\mathcal{K}$ samples to test the accuracy. This is done to keep the right shape of the labels.

In [ ]:
Known_data_X_test = Known_and_Ignored_data_X_test[:-100*5]
Known_data_X_test_label = Known_and_Ignored_data_X_test_label[:-100*5]
print("Shape of the known and ignored data, reference:", Known_data_X_test.shape)
print("Shape of the known and ignored labels, reference:", Known_data_X_test_label.shape)

Shape of the known and ignored data, reference: (1000, 1000)
Shape of the known and ignored labels, reference: (1000, 11)


In [ ]:
Known_data_X_test_label_int=np.argmax(Known_data_X_test_label)

# Defining the $\mathcal{K}$ labels

In [ ]:
KnownLabels = []
# 0 - 5
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 0*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 1*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 2*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 3*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 4*2000])

# 6 - 10
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 5*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 6*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 7*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 8*2000])
KnownLabels.append(Known_and_Ignored_data_X_train_label[1 + 9*2000])

Ignored_label = Known_and_Ignored_data_X_train_label[1 + 10*2000]

In [ ]:
KnownLabels

[array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.], dtype=float32)]

# The ignored label $\mathcal{I}$ is:

In [ ]:
Ignored_label

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

# Never seen before samples, $\mathcal{N}$, first seen during the test: 0 - 500, 700 - 900, 1900 - 2100, 2500 - 2600.

In [ ]:
indices = [slice(0, 500),slice(700, 900),slice(1900, 2100),slice(2500, 2600)]
NeverSeen_data_X_test = np.concatenate([data_X_test[idx, :] for idx in indices], axis=0)

print("Shape of the NeverSeen data, test:", NeverSeen_data_X_test.shape)

Shape of the NeverSeen data, test: (1000, 1000)


# Data for finetuning

In [ ]:
# Finetuning
data_X_finetune = np.load("/content/gdrive/MyDrive/Stanford_data/X_finetune.npy")
# Test data
data_X_test = np.load("/content/gdrive/MyDrive/Stanford_data/X_test.npy")

data_y_finetune = np.load("/content/gdrive/MyDrive/Stanford_data/y_finetune.npy")
# Test labels
data_y_test = np.load("/content/gdrive/MyDrive/Stanford_data/y_test.npy")

data_y_finetune_int = []

for i in range(data_y_finetune.shape[0]):
  data_y_finetune_int.append(int(data_y_finetune[i]))

data_y_test_int = []

for i in range(data_y_test.shape[0]):
  data_y_test_int.append(int(data_y_test[i]))

train_label = tf.keras.utils.to_categorical(data_y_finetune_int)
test_label = tf.keras.utils.to_categorical(data_y_test_int)

#data_X_finetune, train_label = shuffle(data_X_finetune, train_label)

# Known data, $\mathcal{K}$, for the finetuning: 900 - 1300, 1700 - 1900, 2200 - 2500, 2800 - 2900. Ignored samples, $\mathcal{I}$, finetuning: 1400 - 1700, 2600 - 2800.

In [ ]:
indices =[slice(900, 1300),slice(1700, 1900),slice(2200, 2500),slice(2800, 2900),slice(1400, 1700),slice(2600, 2800)]

Known_and_Ignored_data_X_finetune = np.concatenate([data_X_finetune[idx, :] for idx in indices], axis=0)

Known_and_Ignored_data_X_finetune_label_int = []

for i in range(100*10):
  Known_and_Ignored_data_X_finetune_label_int.append(int(data_y_finetune[i]))

for i in range(100*5):
  Known_and_Ignored_data_X_finetune_label_int.append(int(data_y_finetune[100*10]))

Known_and_Ignored_data_X_finetune_label = tf.keras.utils.to_categorical(Known_and_Ignored_data_X_finetune_label_int)

print("Shape of the known and ignored data, finetune:", Known_and_Ignored_data_X_finetune.shape)
print("Shape of the known and ignored labels, finetune:", Known_and_Ignored_data_X_finetune_label.shape)

Shape of the known and ignored data, finetune: (1500, 1000)
Shape of the known and ignored labels, finetune: (1500, 11)


# Uploading the libraries

In [ ]:
# Create a checkpoint directory to store the checkpoints.
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")

import numpy as np
import pandas as pd
import seaborn as sns
from numpy import interp
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
#from sklearn.metrics import mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, roc_curve, plot_roc_curve
from sklearn.metrics import confusion_matrix, classification_report, auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D, concatenate, BatchNormalization, Activation, add
from keras.layers import Conv2D, MaxPooling2D, Reshape, Flatten, Dense, GlobalAveragePooling1D, GlobalMaxPooling1D, Multiply, Conv1DTranspose, LeakyReLU, Dropout
from keras.models import Model, model_from_json
#from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
sns.set_theme(style="whitegrid")

# Defining our model

In [ ]:
# Constants
NoF = 64  # Number of filters in the first convolutional layer
SE_RATIO = 16  # Squeeze-and-Excitation ratio

initializer = tf.keras.initializers.GlorotUniform(seed=0)
initializer2 = tf.keras.initializers.HeUniform(seed=0)

def squeeze_excitation_block(X, ratio=16):
    num_channels = X.shape[-1]
    se = tf.keras.layers.GlobalAveragePooling1D()(X)
    se = tf.keras.layers.Reshape((1, num_channels))(se)
    se = tf.keras.layers.Dense(num_channels // ratio, activation='relu', kernel_initializer=initializer)(se)
    se = tf.keras.layers.Dense(num_channels, activation='sigmoid', kernel_initializer=initializer)(se)
    return tf.keras.layers.Multiply()([X, se])


from tensorflow.keras.layers import Conv1D, BatchNormalization, Activation, Add
from tensorflow.keras import models, layers
from tensorflow.keras import regularizers

initializer = tf.keras.initializers.GlorotUniform(seed=0)
initializer2 = tf.keras.initializers.HeUniform(seed=0)
Stride=1

def identity_block(X, f, filters, stage, block):

  #defining name basis
  ConvNameBase = 'res' + str(stage) + block + '_branch'
  BatchNormBase = 'bn' + str(stage) + block + '_branch'

  #retrieve filters
  F1,F2,F3 = filters

  #save the input value. You'll need this later to add back the main path
  x_shortcut = X

  ### First component of the main path ###
  X = tf.keras.layers.Conv1D(filters=F1, kernel_size=1, strides=Stride, padding='same',
             kernel_initializer=initializer#, name=ConvNameBase
             )(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                         )(X)
  X = tf.keras.layers.Activation('relu')(X)

  ### Second component of main path ###
  X = tf.keras.layers.Conv1D(filters=F2, kernel_size=f, strides=Stride, padding='same',
             kernel_initializer=initializer, #name=ConvNameBase
             )(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                         )(X)
  X = tf.keras.layers.Activation('relu')(X)

  #Third Component of main path
  X = tf.keras.layers.Conv1D(filters=F3, kernel_size=1, strides=Stride, padding='same',
             kernel_initializer=initializer#,name=ConvNameBase
             )(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                         )(X)

  #Final step: add shortcut to the main path, and pass it through ReLU activation
  X = tf.keras.layers.Add()([X, x_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X

def convolutional_block(X, f, filters, stage, block, s=2):

  #Defining name bases
  ConvNameBase = 'res' + str(stage) + block + '_branch'
  BatchNormBase = 'res' + str(stage) + block + '_branch'

  #retrive n_filters
  F1, F2, F3 = filters

  #Save the input value
  x_shortcut = X

  #First component of the main path
  X = tf.keras.layers.Conv1D(F1, strides=s, kernel_size=f, kernel_initializer=initializer,
             padding='same'#, name=ConvNameBase
             )(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                         )(X)
  X = tf.keras.layers.Activation('relu')(X)

  #Second Component of main path
  X = tf.keras.layers.Conv1D(filters=F2, kernel_size=f, strides=Stride, padding='same',
             kernel_initializer=initializer#, name=ConvNameBase
             )(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                         )(X)
  X = tf.keras.layers.Activation('relu')(X)

  #Third component of main path
  X = tf.keras.layers.Conv1D(filters=F3, kernel_size=f, strides=Stride, padding='same',
             kernel_initializer=initializer, name=ConvNameBase)(X)
  X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
  )(X)

  ###### SHORTCUT PATH ######
  x_shortcut = tf.keras.layers.Conv1D(filters = F3, kernel_size=1, strides=s,
                      padding='same', #name=ConvNameBase,
                      kernel_initializer=initializer)(x_shortcut)
  x_shortcut = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,
                                  )(x_shortcut)

  #Add shortcut to main path and pass in through ReLU activation
  X = tf.keras.layers.Add()([X, x_shortcut])
  X = tf.keras.layers.Activation('relu')(X)

  return X

def create_model():
    input_shape = (1000, 1)
    nb_classes = 10 + 1

    x_input = tf.keras.layers.Input(input_shape)
    X = tf.keras.layers.ZeroPadding1D(padding=3)(x_input)

    X = tf.keras.layers.Conv1D(NoF, kernel_size=7, strides=2, name='Convolution1',
                               kernel_initializer=initializer)(X)
    X = tf.keras.layers.BatchNormalization(name="BatchNormStage1")(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPooling1D(3, strides=2)(X)

    X = convolutional_block(X, f=3, filters=[NoF, NoF, NoF * 4], stage=2, block='a', s=1)
    X = identity_block(X, 3, [NoF, NoF, NoF * 4], stage=2, block='b')

    X = convolutional_block(X, f=3, filters=[NoF * 2, NoF * 2, NoF * 8], stage=3, block='a')
    X = identity_block(X, 3, [NoF * 2, NoF * 2, NoF * 8], stage=3, block='b')

    X = convolutional_block(X, f=3, filters=[NoF * 4, NoF * 4, NoF * 16], stage=4, block='a')
    X = identity_block(X, 3, [NoF * 4, NoF * 4, NoF * 16], stage=4, block='b')

    X = convolutional_block(X, f=3, filters=[NoF * 4, NoF * 4, NoF * 32], stage=5, block='a')
    X = identity_block(X, f=3, filters=[NoF * 4, NoF * 4, NoF * 32], stage=5, block='b')

    # Applying SE mechanism before transposed convolutional layers
    X = squeeze_excitation_block(X)

    # Adding transposed convolutional layers
    X = Conv1DTranspose(filters=NoF, kernel_size=3, strides=2, padding='same', kernel_initializer=initializer)(X)
    X = tf.keras.layers.BatchNormalization(axis=2, momentum=0.99, trainable=False,)(X)
    X = LeakyReLU(alpha=0.2)(X)

    X = Flatten()(X)

    # Adding dropout regularization
    X = Dropout(0.5)(X)

    X = Dense(nb_classes, activation='softmax', kernel_initializer=initializer)(X)

    res_net = models.Model(inputs=x_input, outputs=X, name='ResNet29')

    return res_net

model = create_model()
model.summary()

Model: "ResNet29"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 1000, 1)]            0         []                            
                                                                                                  
 zero_padding1d (ZeroPaddin  (None, 1006, 1)              0         ['input_1[0][0]']             
 g1D)                                                                                             
                                                                                                  
 Convolution1 (Conv1D)       (None, 500, 64)              512       ['zero_padding1d[0][0]']      
                                                                                                  
 BatchNormStage1 (BatchNorm  (None, 500, 64)              256       ['Convolution1[0][0]'] 

In [ ]:
with strategy.scope():

  # By N_classes, I mean classes of interest.
  N_classes = 10

  def Open_set_categorical_crossentropy_trial_total(target,output):
    def Open_set_categorical_crossentropy_trial_first(target,output):
      output = tf.convert_to_tensor(output)
      target = tf.cast(target, target.dtype)

      epsilon = tf.keras.backend.epsilon()
      output = tf.clip_by_value(output, epsilon, 1. - epsilon)

      Sum = tf.math.reduce_sum(output,axis=-1,keepdims=True)

      output = tf.math.divide(output, Sum)

      Checker = tf.math.reduce_sum(tf.math.multiply(Ignored_label, target),axis=-1)
      One = tf.math.reduce_sum(tf.math.multiply(Ignored_label, Ignored_label),axis=-1)

      output = - tf.math.reduce_sum(tf.math.multiply(target,tf.math.log(output)),axis=-1)
      output = tf.math.multiply(tf.math.subtract(One,Checker),output)

      return output

    def Open_set_categorical_crossentropy_trial_second(target,output):
      output = tf.convert_to_tensor(output)
      target = tf.cast(target, target.dtype)

      epsilon = tf.keras.backend.epsilon()
      output = tf.clip_by_value(output, epsilon, 1. - epsilon)

      Sum = tf.math.reduce_sum(output,axis=-1,keepdims=True)

      output = tf.math.divide(output, Sum)

      Checker = tf.math.reduce_sum(tf.math.multiply(Ignored_label, target),axis=-1)
      One = tf.math.reduce_sum(tf.math.multiply(Ignored_label, Ignored_label),axis=-1)
      Zero = tf.math.reduce_sum(tf.math.multiply(KnownLabels[1], KnownLabels[2]),axis=-1)

      for i in range(N_classes):
        addon = - tf.math.reduce_sum(tf.math.multiply(KnownLabels[i], tf.math.log(output)),axis=-1)
        addon = tf.math.divide(addon,N_classes)
        Zero = tf.math.add(addon,Zero)
      Zero = tf.math.multiply(Checker, Zero)

      return Zero

    output1 = Open_set_categorical_crossentropy_trial_first(target,output)
    output2 = Open_set_categorical_crossentropy_trial_second(target,output)
    output3 = Open_set_categorical_crossentropy_trial_second(target,output)

    output = tf.math.add(output1,output2)
    output = tf.math.add(output,output3)

    return output

  res_net = create_model()

  optimizer = tf.keras.optimizers.Adam(learning_rate=.00001)

  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=res_net)

  callbacks = [EarlyStopping(monitor='val_loss', patience=20, mode='min'), ModelCheckpoint('/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
  res_net.compile(loss=Open_set_categorical_crossentropy_trial_total,optimizer=tf.keras.optimizers.Adam(learning_rate=.00001))

  Known_and_Ignored_data_X_reference, Known_and_Ignored_data_X_train_label = shuffle(Known_and_Ignored_data_X_reference,Known_and_Ignored_data_X_train_label)

  history = res_net.fit(Known_and_Ignored_data_X_reference, Known_and_Ignored_data_X_train_label, epochs=200, batch_size=32, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)


Streaming output truncated to the last 5000 lines.
Epoch 50/200
749/750 [============================>.] - ETA: 0s - loss: 1.5700
Epoch 50: val_loss did not improve from 1.59729
750/750 [==============================] - 20s 26ms/step - loss: 1.5702 - val_loss: 1.6010
Epoch 51/200
748/750 [============================>.] - ETA: 0s - loss: 1.5701
Epoch 51: val_loss improved from 1.59729 to 1.59556, saving model to /content/gdrive/MyDrive/Stanford_data/05_EOS_K_p1_I_p2.h5
750/750 [==============================] - 23s 30ms/step - loss: 1.5708 - val_loss: 1.5956
Epoch 52/200
749/750 [============================>.] - ETA: 0s - loss: 1.5695
Epoch 52: val_loss did not improve from 1.59556
750/750 [==============================] - 20s 27ms/step - loss: 1.5700 - val_loss: 1.6064
Epoch 53/200
750/750 [==============================] - ETA: 0s - loss: 1.5663
Epoch 53: val_loss did not improve from 1.59556
750/750 [==============================] - 19s 26ms/step - loss: 1.5663 - val_loss: 1.598

In [ ]:
# Now run and see the models with the best validation accuracy

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 01 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/02_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 02 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/03_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 03 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/04_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 04 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/05_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 05 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/06_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 06 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/07_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 07 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/08_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 08 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/09_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 09 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/10_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 10 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/11_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 11 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/12_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 12 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/13_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 13 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/14_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 14 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/15_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 15 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/16_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 16 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/17_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 17 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/18_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 18 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/19_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 19 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/20_EOS_K_p1_I_p2.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 20 run:', test_acc)

32/32 [==============================] - 6s 110ms/step - loss: 2.3262 - accuracy: 0.5570
Test accuracy, 01 run: 0.5569999814033508
32/32 [==============================] - 6s 112ms/step - loss: 2.5075 - accuracy: 0.5210
Test accuracy, 02 run: 0.5210000276565552
32/32 [==============================] - 6s 118ms/step - loss: 2.8250 - accuracy: 0.5620
Test accuracy, 03 run: 0.5619999766349792
32/32 [==============================] - 6s 116ms/step - loss: 2.4159 - accuracy: 0.5740
Test accuracy, 04 run: 0.5740000009536743
32/32 [==============================] - 6s 112ms/step - loss: 2.4762 - accuracy: 0.5800
Test accuracy, 05 run: 0.5799999833106995
32/32 [==============================] - 5s 112ms/step - loss: 2.9371 - accuracy: 0.5410
Test accuracy, 06 run: 0.5410000085830688
32/32 [==============================] - 6s 104ms/step - loss: 2.0138 - accuracy: 0.5780
Test accuracy, 07 run: 0.578000009059906
32/32 [==============================] - 6s 111ms/step - loss: 2.8400 - accuracy: 0.

In [ ]:
with strategy.scope():

  # By N_classes, I mean classes of interest.
  N_classes = 10

  def Open_set_categorical_crossentropy_trial_total(target,output):
    def Open_set_categorical_crossentropy_trial_first(target,output):
      output = tf.convert_to_tensor(output)
      target = tf.cast(target, target.dtype)

      epsilon = tf.keras.backend.epsilon()
      output = tf.clip_by_value(output, epsilon, 1. - epsilon)

      Sum = tf.math.reduce_sum(output,axis=-1,keepdims=True)

      output = tf.math.divide(output, Sum)

      Checker = tf.math.reduce_sum(tf.math.multiply(Ignored_label, target),axis=-1)
      One = tf.math.reduce_sum(tf.math.multiply(Ignored_label, Ignored_label),axis=-1)

      output = - tf.math.reduce_sum(tf.math.multiply(target,tf.math.log(output)),axis=-1)
      output = tf.math.multiply(tf.math.subtract(One,Checker),output)

      return output

    def Open_set_categorical_crossentropy_trial_second(target,output):
      output = tf.convert_to_tensor(output)
      target = tf.cast(target, target.dtype)

      epsilon = tf.keras.backend.epsilon()
      output = tf.clip_by_value(output, epsilon, 1. - epsilon)

      Sum = tf.math.reduce_sum(output,axis=-1,keepdims=True)

      output = tf.math.divide(output, Sum)

      Checker = tf.math.reduce_sum(tf.math.multiply(Ignored_label, target),axis=-1)
      One = tf.math.reduce_sum(tf.math.multiply(Ignored_label, Ignored_label),axis=-1)
      Zero = tf.math.reduce_sum(tf.math.multiply(KnownLabels[1], KnownLabels[2]),axis=-1)

      for i in range(N_classes):
        addon = - tf.math.reduce_sum(tf.math.multiply(KnownLabels[i], tf.math.log(output)),axis=-1)
        addon = tf.math.divide(addon,N_classes)
        Zero = tf.math.add(addon,Zero)
      Zero = tf.math.multiply(Checker, Zero)

      return Zero

    output1 = Open_set_categorical_crossentropy_trial_first(target,output)
    output2 = Open_set_categorical_crossentropy_trial_second(target,output)
    output3 = Open_set_categorical_crossentropy_trial_second(target,output)

    output = tf.math.add(output1,output2)
    output = tf.math.add(output,output3)

    return output

  res_net = create_model()

  optimizer = tf.keras.optimizers.Adam(learning_rate=.00001/10)

  checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=res_net)

  callbacks = [EarlyStopping(monitor='val_loss', patience=30, mode='min'), ModelCheckpoint('/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2_FTm3.h5', verbose=1, monitor='val_loss', save_best_only=True, mode='min')]
  res_net.compile(loss=Open_set_categorical_crossentropy_trial_total,optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10))

  Known_and_Ignored_data_X_finetune,Known_and_Ignored_data_X_finetune_label = shuffle(Known_and_Ignored_data_X_finetune,Known_and_Ignored_data_X_finetune_label)

  res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2.h5")

  for layer in res_net.layers[:-3]:
    layer.trainable = False

  history = res_net.fit(Known_and_Ignored_data_X_finetune,Known_and_Ignored_data_X_finetune_label, epochs=500, batch_size=32, verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks)

  res_net.save_weights('/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2_FTm3_PP.h5')


Streaming output truncated to the last 5000 lines.
Epoch 137: val_loss improved from 1.67579 to 1.67496, saving model to /content/gdrive/MyDrive/Stanford_data/09_EOS_K_p1_I_p2_FTm3.h5
38/38 [==============================] - 4s 105ms/step - loss: 1.6068 - val_loss: 1.6750
Epoch 138/500
36/38 [===========================>..] - ETA: 0s - loss: 1.6166
Epoch 138: val_loss did not improve from 1.67496
38/38 [==============================] - 2s 46ms/step - loss: 1.6012 - val_loss: 1.6756
Epoch 139/500
37/38 [============================>.] - ETA: 0s - loss: 1.5986
Epoch 139: val_loss did not improve from 1.67496
38/38 [==============================] - 2s 45ms/step - loss: 1.6010 - val_loss: 1.6756
Epoch 140/500
36/38 [===========================>..] - ETA: 0s - loss: 1.5913
Epoch 140: val_loss improved from 1.67496 to 1.67233, saving model to /content/gdrive/MyDrive/Stanford_data/09_EOS_K_p1_I_p2_FTm3.h5
38/38 [==============================] - 5s 121ms/step - loss: 1.6028 - val_loss: 1.67

TypeError: ignored

In [ ]:
# Now run and see the models with the best validation accuracy

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 01 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/02_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 02 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/03_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 03 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/04_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 04 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/05_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 05 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/06_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 06 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/07_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 07 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/08_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 08 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/09_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 09 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/10_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 10 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/11_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 11 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/12_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 12 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/13_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 13 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/14_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 14 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/15_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 15 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/16_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 16 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/17_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 17 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/18_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 18 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/19_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 19 run:', test_acc)

res_net = create_model()
res_net.load_weights("/content/gdrive/MyDrive/Stanford_data/20_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net.compile(loss="categorical_crossentropy",optimizer=tf.keras.optimizers.Adam(learning_rate=.00001/10),
            metrics=['accuracy'])
test_loss, test_acc = res_net.evaluate(Known_data_X_test,Known_data_X_test_label)
print('Test accuracy, 20 run:', test_acc)

32/32 [==============================] - 34s 847ms/step - loss: 0.3766 - accuracy: 0.9040
Test accuracy, 01 run: 0.9039999842643738
32/32 [==============================] - 30s 863ms/step - loss: 0.3887 - accuracy: 0.8960
Test accuracy, 02 run: 0.8960000276565552
32/32 [==============================] - 26s 759ms/step - loss: 0.4740 - accuracy: 0.8960
Test accuracy, 03 run: 0.8960000276565552
32/32 [==============================] - 29s 808ms/step - loss: 0.3669 - accuracy: 0.9130
Test accuracy, 04 run: 0.9129999876022339
32/32 [==============================] - 27s 781ms/step - loss: 0.4233 - accuracy: 0.8880
Test accuracy, 05 run: 0.8880000114440918
32/32 [==============================] - 27s 749ms/step - loss: 0.4386 - accuracy: 0.8850
Test accuracy, 06 run: 0.8849999904632568
32/32 [==============================] - 31s 873ms/step - loss: 0.3669 - accuracy: 0.9020
Test accuracy, 07 run: 0.9020000100135803
32/32 [==============================] - 30s 860ms/step - loss: 0.4093 - acc

# Check, how does this approach work in an ***open*** world

In [ ]:
res_net01 = create_model()
res_net02 = create_model()
res_net03 = create_model()
res_net04 = create_model()
res_net05 = create_model()
res_net06 = create_model()
res_net07 = create_model()
res_net08 = create_model()
res_net09 = create_model()
res_net10 = create_model()
res_net11 = create_model()
res_net12 = create_model()
res_net13 = create_model()
res_net14 = create_model()
res_net15 = create_model()
res_net16 = create_model()
res_net17 = create_model()
res_net18 = create_model()
res_net19 = create_model()
res_net20 = create_model()

res_net01.load_weights("/content/gdrive/MyDrive/Stanford_data/01_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net02.load_weights("/content/gdrive/MyDrive/Stanford_data/02_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net03.load_weights("/content/gdrive/MyDrive/Stanford_data/03_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net04.load_weights("/content/gdrive/MyDrive/Stanford_data/04_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net05.load_weights("/content/gdrive/MyDrive/Stanford_data/05_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net06.load_weights("/content/gdrive/MyDrive/Stanford_data/06_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net07.load_weights("/content/gdrive/MyDrive/Stanford_data/07_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net08.load_weights("/content/gdrive/MyDrive/Stanford_data/08_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net09.load_weights("/content/gdrive/MyDrive/Stanford_data/09_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net10.load_weights("/content/gdrive/MyDrive/Stanford_data/10_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net11.load_weights("/content/gdrive/MyDrive/Stanford_data/11_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net12.load_weights("/content/gdrive/MyDrive/Stanford_data/12_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net13.load_weights("/content/gdrive/MyDrive/Stanford_data/13_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net14.load_weights("/content/gdrive/MyDrive/Stanford_data/14_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net15.load_weights("/content/gdrive/MyDrive/Stanford_data/15_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net16.load_weights("/content/gdrive/MyDrive/Stanford_data/16_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net17.load_weights("/content/gdrive/MyDrive/Stanford_data/17_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net18.load_weights("/content/gdrive/MyDrive/Stanford_data/18_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net19.load_weights("/content/gdrive/MyDrive/Stanford_data/19_EOS_K_p1_I_p2_FTm3_PP.h5")
res_net20.load_weights("/content/gdrive/MyDrive/Stanford_data/20_EOS_K_p1_I_p2_FTm3_PP.h5")



prediction01_known = res_net01.predict(Known_data_X_test)
prediction02_known = res_net02.predict(Known_data_X_test)
prediction03_known = res_net03.predict(Known_data_X_test)
prediction04_known = res_net04.predict(Known_data_X_test)
prediction05_known = res_net05.predict(Known_data_X_test)
prediction06_known = res_net06.predict(Known_data_X_test)
prediction07_known = res_net07.predict(Known_data_X_test)
prediction08_known = res_net08.predict(Known_data_X_test)
prediction09_known = res_net09.predict(Known_data_X_test)
prediction10_known = res_net10.predict(Known_data_X_test)

prediction11_known = res_net11.predict(Known_data_X_test)
prediction12_known = res_net12.predict(Known_data_X_test)
prediction13_known = res_net13.predict(Known_data_X_test)
prediction14_known = res_net14.predict(Known_data_X_test)
prediction15_known = res_net15.predict(Known_data_X_test)
prediction16_known = res_net16.predict(Known_data_X_test)
prediction17_known = res_net17.predict(Known_data_X_test)
prediction18_known = res_net18.predict(Known_data_X_test)
prediction19_known = res_net19.predict(Known_data_X_test)
prediction20_known = res_net20.predict(Known_data_X_test)


32/32 [==============================] - 17s 508ms/step


In [ ]:
prediction01_unknown = res_net01.predict(NeverSeen_data_X_test)
prediction02_unknown = res_net02.predict(NeverSeen_data_X_test)
prediction03_unknown = res_net03.predict(NeverSeen_data_X_test)
prediction04_unknown = res_net04.predict(NeverSeen_data_X_test)
prediction05_unknown = res_net05.predict(NeverSeen_data_X_test)
prediction06_unknown = res_net06.predict(NeverSeen_data_X_test)
prediction07_unknown = res_net07.predict(NeverSeen_data_X_test)
prediction08_unknown = res_net08.predict(NeverSeen_data_X_test)
prediction09_unknown = res_net09.predict(NeverSeen_data_X_test)
prediction10_unknown = res_net10.predict(NeverSeen_data_X_test)
prediction11_unknown = res_net11.predict(NeverSeen_data_X_test)
prediction12_unknown = res_net12.predict(NeverSeen_data_X_test)
prediction13_unknown = res_net13.predict(NeverSeen_data_X_test)
prediction14_unknown = res_net14.predict(NeverSeen_data_X_test)
prediction15_unknown = res_net15.predict(NeverSeen_data_X_test)
prediction16_unknown = res_net16.predict(NeverSeen_data_X_test)
prediction17_unknown = res_net17.predict(NeverSeen_data_X_test)
prediction18_unknown = res_net18.predict(NeverSeen_data_X_test)
prediction19_unknown = res_net19.predict(NeverSeen_data_X_test)
prediction20_unknown = res_net20.predict(NeverSeen_data_X_test)


prediction_unknown_ensemble_1 = (prediction01_unknown + prediction02_unknown + prediction03_unknown + prediction04_unknown + prediction05_unknown)/5
prediction_unknown_ensemble_2 = (prediction06_unknown + prediction07_unknown + prediction08_unknown + prediction09_unknown + prediction10_unknown)/5
prediction_unknown_ensemble_3 = (prediction11_unknown + prediction12_unknown + prediction13_unknown + prediction14_unknown + prediction15_unknown)/5
prediction_unknown_ensemble_4 = (prediction16_unknown + prediction17_unknown + prediction18_unknown + prediction19_unknown + prediction20_unknown)/5

32/32 [==============================] - 15s 476ms/step


In [ ]:
Known_data_X_test_label_int = []
for i in range(Known_data_X_test_label.shape[0]):
  Known_data_X_test_label_int.append(np.argmax(Known_data_X_test_label[i]))

In [ ]:
prediction_known_ensemble_1 = (prediction01_known + prediction02_known + prediction03_known + prediction04_known + prediction05_known)/5

true = 0
for i in range(Known_data_X_test_label.shape[0]):
  if prediction_known_ensemble_1.argmax(axis=1)[i] == Known_data_X_test_label_int[i]:
    true += 1
print("Accuracy of the first ensemble on the knowns", true/(Known_data_X_test_label.shape[0]))

prediction_known_ensemble_2 = (prediction06_known + prediction07_known + prediction08_known + prediction09_known + prediction10_known)/5

true = 0
for i in range(Known_data_X_test_label.shape[0]):
  if prediction_known_ensemble_2.argmax(axis=1)[i] == Known_data_X_test_label_int[i]:
    true += 1
print("Accuracy of the second ensemble on the knowns", true/(Known_data_X_test_label.shape[0]))

prediction_known_ensemble_3 = (prediction11_known + prediction12_known + prediction13_known + prediction14_known + prediction15_known)/5

true = 0
for i in range(Known_data_X_test_label.shape[0]):
  if prediction_known_ensemble_3.argmax(axis=1)[i] == Known_data_X_test_label_int[i]:
    true += 1
print("Accuracy of the third ensemble on the knowns", true/(Known_data_X_test_label.shape[0]))

prediction_known_ensemble_4 = (prediction16_known + prediction17_known + prediction18_known + prediction19_known + prediction20_known)/5

true = 0
for i in range(Known_data_X_test_label.shape[0]):
  if prediction_known_ensemble_4.argmax(axis=1)[i] == Known_data_X_test_label_int[i]:
    true += 1
print("Accuracy of the fourth ensemble on the knowns", true/(Known_data_X_test_label.shape[0]))

Accuracy of the first ensemble on the knowns 0.901
Accuracy of the second ensemble on the knowns 0.897
Accuracy of the third ensemble on the knowns 0.905
Accuracy of the fourth ensemble on the knowns 0.904
Accuracy of the first ensemble on the knowns 0.901
Accuracy of the second ensemble on the knowns 0.897
Accuracy of the third ensemble on the knowns 0.905
Accuracy of the fourth ensemble on the knowns 0.904


# Separating $\mathcal{K}$ and $\mathcal{N}$ by a fixed (class-independendent) threshold

In [ ]:
prediction_known = prediction_known_ensemble_2
prediction_unknown = prediction_unknown_ensemble_2

In [ ]:
def thresholding(threshold):
  true = 0
  for i in range(prediction_known.shape[0]):
    if prediction_known.argmax(axis=1)[i] == Known_data_X_test_label_int[i] and max(prediction_known[i]) > threshold:
      true += 1
  return true/(prediction_known.shape[0])

In [ ]:
CorrectThresholding = []
for i in range(90):
  CorrectThresholding.append([0.1 + 0.01*i,100*thresholding(0.1 + 0.01*i)])
for i in range(10):
  CorrectThresholding.append([0.99 + 0.001*i,100*thresholding(0.99 + 0.001*i)])
CorrectThresholding

[[0.1, 89.7],
 [0.11, 89.7],
 [0.12000000000000001, 89.7],
 [0.13, 89.5],
 [0.14, 89.5],
 [0.15000000000000002, 89.3],
 [0.16, 89.2],
 [0.17, 89.1],
 [0.18, 89.1],
 [0.19, 89.1],
 [0.2, 89.1],
 [0.21000000000000002, 89.1],
 [0.22, 89.1],
 [0.23, 88.9],
 [0.24000000000000002, 88.9],
 [0.25, 88.8],
 [0.26, 88.8],
 [0.27, 88.8],
 [0.28, 88.8],
 [0.29000000000000004, 88.8],
 [0.30000000000000004, 88.7],
 [0.31, 88.6],
 [0.32, 88.6],
 [0.33, 88.6],
 [0.33999999999999997, 88.6],
 [0.35, 88.6],
 [0.36, 88.5],
 [0.37, 88.4],
 [0.38, 88.3],
 [0.39, 88.1],
 [0.4, 88.0],
 [0.41000000000000003, 87.9],
 [0.42000000000000004, 87.8],
 [0.43000000000000005, 87.8],
 [0.44000000000000006, 87.8],
 [0.45000000000000007, 87.7],
 [0.45999999999999996, 87.4],
 [0.47, 87.4],
 [0.48, 87.4],
 [0.49, 87.0],
 [0.5, 86.8],
 [0.51, 86.8],
 [0.52, 86.7],
 [0.53, 86.5],
 [0.54, 85.8],
 [0.55, 85.7],
 [0.56, 85.39999999999999],
 [0.5700000000000001, 85.2],
 [0.58, 85.1],
 [0.59, 84.89999999999999],
 [0.6, 84.899999999

In [ ]:
def false_identify(threshold):
  false_identify = 0
  for i in range(prediction_known.shape[0]):
    if prediction_known.argmax(axis=1)[i] != Known_data_X_test_label_int[i] and max(prediction_known[i]) > threshold:
      false_identify += 1
  return false_identify/(prediction_known.shape[0])

In [ ]:
FalseIdentify = []
for i in range(90):
  FalseIdentify.append([0.1 + 0.01*i,100*false_identify(0.1 + 0.01*i)])
for i in range(10):
  FalseIdentify.append([0.99 + 0.001*i,100*false_identify(0.99 + 0.001*i)])
FalseIdentify

[[0.1, 10.299999999999999],
 [0.11, 10.299999999999999],
 [0.12000000000000001, 10.100000000000001],
 [0.13, 10.100000000000001],
 [0.14, 10.0],
 [0.15000000000000002, 10.0],
 [0.16, 10.0],
 [0.17, 10.0],
 [0.18, 10.0],
 [0.19, 10.0],
 [0.2, 9.9],
 [0.21000000000000002, 9.9],
 [0.22, 9.8],
 [0.23, 9.8],
 [0.24000000000000002, 9.8],
 [0.25, 9.8],
 [0.26, 9.8],
 [0.27, 9.8],
 [0.28, 9.8],
 [0.29000000000000004, 9.8],
 [0.30000000000000004, 9.700000000000001],
 [0.31, 9.700000000000001],
 [0.32, 9.6],
 [0.33, 9.6],
 [0.33999999999999997, 9.3],
 [0.35, 9.3],
 [0.36, 9.3],
 [0.37, 9.2],
 [0.38, 9.0],
 [0.39, 8.799999999999999],
 [0.4, 8.799999999999999],
 [0.41000000000000003, 8.6],
 [0.42000000000000004, 8.6],
 [0.43000000000000005, 8.5],
 [0.44000000000000006, 8.3],
 [0.45000000000000007, 8.3],
 [0.45999999999999996, 8.200000000000001],
 [0.47, 8.1],
 [0.48, 8.1],
 [0.49, 7.9],
 [0.5, 7.7],
 [0.51, 7.5],
 [0.52, 7.3999999999999995],
 [0.53, 7.3],
 [0.54, 7.1],
 [0.55, 7.000000000000001],


In [ ]:
def misclassifyunknown(threshold):
  misclassifyunknown = 0
  for i in range(prediction_unknown.shape[0]):
    if max(prediction_unknown[i]) > threshold:
      misclassifyunknown += 1
  return misclassifyunknown/(prediction_unknown.shape[0])

In [ ]:
UnknownMisclassify = []
for i in range(90):
  UnknownMisclassify.append([0.1 + 0.01*i,100*misclassifyunknown(0.1 + 0.01*i)])
for i in range(10):
  UnknownMisclassify.append([0.99 + 0.001*i,100*misclassifyunknown(0.99 + 0.001*i)])
UnknownMisclassify

[[0.1, 100.0],
 [0.11, 100.0],
 [0.12000000000000001, 100.0],
 [0.13, 99.7],
 [0.14, 99.2],
 [0.15000000000000002, 99.1],
 [0.16, 98.7],
 [0.17, 98.1],
 [0.18, 97.5],
 [0.19, 96.5],
 [0.2, 95.89999999999999],
 [0.21000000000000002, 95.1],
 [0.22, 93.4],
 [0.23, 92.80000000000001],
 [0.24000000000000002, 92.10000000000001],
 [0.25, 91.4],
 [0.26, 90.8],
 [0.27, 89.7],
 [0.28, 89.3],
 [0.29000000000000004, 87.9],
 [0.30000000000000004, 87.2],
 [0.31, 86.2],
 [0.32, 85.39999999999999],
 [0.33, 84.6],
 [0.33999999999999997, 83.7],
 [0.35, 83.39999999999999],
 [0.36, 82.19999999999999],
 [0.37, 81.6],
 [0.38, 80.5],
 [0.39, 79.60000000000001],
 [0.4, 79.10000000000001],
 [0.41000000000000003, 78.2],
 [0.42000000000000004, 77.4],
 [0.43000000000000005, 76.9],
 [0.44000000000000006, 75.6],
 [0.45000000000000007, 74.6],
 [0.45999999999999996, 73.5],
 [0.47, 72.5],
 [0.48, 71.89999999999999],
 [0.49, 70.8],
 [0.5, 69.69999999999999],
 [0.51, 68.7],
 [0.52, 67.9],
 [0.53, 66.60000000000001],
 [0

In [ ]:
def dontknow(threshold):
  notknown = 0
  for i in range(prediction_known.shape[0]):
    if max(prediction_known[i]) <= threshold:
      notknown += 1
  return notknown/(prediction_known.shape[0])

In [ ]:
DontKnowThresholding = []
for i in range(90):
  DontKnowThresholding.append([0.1 + 0.01*i,100*dontknow(0.1 + 0.01*i)])
for i in range(10):
  DontKnowThresholding.append([0.99 + 0.001*i,100*dontknow(0.99 + 0.001*i)])
DontKnowThresholding

[[0.1, 0.0],
 [0.11, 0.0],
 [0.12000000000000001, 0.2],
 [0.13, 0.4],
 [0.14, 0.5],
 [0.15000000000000002, 0.7000000000000001],
 [0.16, 0.8],
 [0.17, 0.8999999999999999],
 [0.18, 0.8999999999999999],
 [0.19, 0.8999999999999999],
 [0.2, 1.0],
 [0.21000000000000002, 1.0],
 [0.22, 1.0999999999999999],
 [0.23, 1.3],
 [0.24000000000000002, 1.3],
 [0.25, 1.4000000000000001],
 [0.26, 1.4000000000000001],
 [0.27, 1.4000000000000001],
 [0.28, 1.4000000000000001],
 [0.29000000000000004, 1.4000000000000001],
 [0.30000000000000004, 1.6],
 [0.31, 1.7000000000000002],
 [0.32, 1.7999999999999998],
 [0.33, 1.7999999999999998],
 [0.33999999999999997, 2.1],
 [0.35, 2.1],
 [0.36, 2.1999999999999997],
 [0.37, 2.4],
 [0.38, 2.7],
 [0.39, 3.1],
 [0.4, 3.2],
 [0.41000000000000003, 3.5000000000000004],
 [0.42000000000000004, 3.5999999999999996],
 [0.43000000000000005, 3.6999999999999997],
 [0.44000000000000006, 3.9],
 [0.45000000000000007, 4.0],
 [0.45999999999999996, 4.3999999999999995],
 [0.47, 4.5],
 [0.48

In [ ]:
aligned_array = [element for row in prediction_unknown for element in row]
max_value = max(aligned_array)

print("The maximum value in the aligned array is:", max_value)


The maximum value in the aligned array is: 0.9999973


In [ ]:
thresholding(max_value)

0.065

In [ ]:
dontknow(max_value)

0.935

In [ ]:
misclassifyunknown(max_value)

0.0

In [ ]:
false_identify(max_value)

0.0

In [ ]:
DontKnowThresholdingANOTHER = []

for i in range(89):
  DontKnowThresholdingANOTHER.append([100*thresholding(0.1 + 0.01*i),100*dontknow(0.1 + 0.01*i),])

for i in range(10):
  DontKnowThresholdingANOTHER.append([100*thresholding(0.99 + 0.001*i),100*dontknow(0.99 + 0.001*i),])

for i in range(10):
  DontKnowThresholdingANOTHER.append([100*thresholding(0.999 + 0.0001*i),100*dontknow(0.999 + 0.0001*i),])

for i in range(10):
  DontKnowThresholdingANOTHER.append([100*thresholding(0.9999 + 0.00001*i),100*dontknow(0.9999 + 0.00001*i),])

for i in range(9):
  DontKnowThresholdingANOTHER.append([100*thresholding(0.99999 + 0.000001*i),100*dontknow(0.99999 + 0.000001*i),])

DontKnowThresholdingANOTHER.append([100*thresholding(max_value),100*dontknow(max_value)])

DontKnowThresholdingANOTHER

[[89.7, 0.0],
 [89.7, 0.0],
 [89.7, 0.2],
 [89.5, 0.4],
 [89.5, 0.5],
 [89.3, 0.7000000000000001],
 [89.2, 0.8],
 [89.1, 0.8999999999999999],
 [89.1, 0.8999999999999999],
 [89.1, 0.8999999999999999],
 [89.1, 1.0],
 [89.1, 1.0],
 [89.1, 1.0999999999999999],
 [88.9, 1.3],
 [88.9, 1.3],
 [88.8, 1.4000000000000001],
 [88.8, 1.4000000000000001],
 [88.8, 1.4000000000000001],
 [88.8, 1.4000000000000001],
 [88.8, 1.4000000000000001],
 [88.7, 1.6],
 [88.6, 1.7000000000000002],
 [88.6, 1.7999999999999998],
 [88.6, 1.7999999999999998],
 [88.6, 2.1],
 [88.6, 2.1],
 [88.5, 2.1999999999999997],
 [88.4, 2.4],
 [88.3, 2.7],
 [88.1, 3.1],
 [88.0, 3.2],
 [87.9, 3.5000000000000004],
 [87.8, 3.5999999999999996],
 [87.8, 3.6999999999999997],
 [87.8, 3.9],
 [87.7, 4.0],
 [87.4, 4.3999999999999995],
 [87.4, 4.5],
 [87.4, 4.5],
 [87.0, 5.1],
 [86.8, 5.5],
 [86.8, 5.7],
 [86.7, 5.8999999999999995],
 [86.5, 6.2],
 [85.8, 7.1],
 [85.7, 7.3],
 [85.39999999999999, 7.8],
 [85.2, 8.200000000000001],
 [85.1, 8.799999

In [ ]:
UnknownMisclassifyANOTHER = []
for i in range(89):
  UnknownMisclassifyANOTHER.append([100*thresholding(0.1 + 0.01*i),100*misclassifyunknown(0.1 + 0.01*i),])
for i in range(10):
  UnknownMisclassifyANOTHER.append([100*thresholding(0.99 + 0.001*i),100*misclassifyunknown(0.99 + 0.001*i),])

for i in range(10):
  UnknownMisclassifyANOTHER.append([100*thresholding(0.999 + 0.0001*i),100*misclassifyunknown(0.999 + 0.0001*i),])

for i in range(10):
  UnknownMisclassifyANOTHER.append([100*thresholding(0.9999 + 0.00001*i),100*misclassifyunknown(0.9999 + 0.00001*i),])

for i in range(9):
  UnknownMisclassifyANOTHER.append([100*thresholding(0.99999 + 0.000001*i),100*misclassifyunknown(0.99999 + 0.000001*i),])

UnknownMisclassifyANOTHER.append([100*thresholding(max_value),100*misclassifyunknown(max_value),])

UnknownMisclassifyANOTHER

[[89.7, 100.0],
 [89.7, 100.0],
 [89.7, 100.0],
 [89.5, 99.7],
 [89.5, 99.2],
 [89.3, 99.1],
 [89.2, 98.7],
 [89.1, 98.1],
 [89.1, 97.5],
 [89.1, 96.5],
 [89.1, 95.89999999999999],
 [89.1, 95.1],
 [89.1, 93.4],
 [88.9, 92.80000000000001],
 [88.9, 92.10000000000001],
 [88.8, 91.4],
 [88.8, 90.8],
 [88.8, 89.7],
 [88.8, 89.3],
 [88.8, 87.9],
 [88.7, 87.2],
 [88.6, 86.2],
 [88.6, 85.39999999999999],
 [88.6, 84.6],
 [88.6, 83.7],
 [88.6, 83.39999999999999],
 [88.5, 82.19999999999999],
 [88.4, 81.6],
 [88.3, 80.5],
 [88.1, 79.60000000000001],
 [88.0, 79.10000000000001],
 [87.9, 78.2],
 [87.8, 77.4],
 [87.8, 76.9],
 [87.8, 75.6],
 [87.7, 74.6],
 [87.4, 73.5],
 [87.4, 72.5],
 [87.4, 71.89999999999999],
 [87.0, 70.8],
 [86.8, 69.69999999999999],
 [86.8, 68.7],
 [86.7, 67.9],
 [86.5, 66.60000000000001],
 [85.8, 65.4],
 [85.7, 64.1],
 [85.39999999999999, 63.0],
 [85.2, 62.3],
 [85.1, 60.8],
 [84.89999999999999, 59.599999999999994],
 [84.89999999999999, 57.99999999999999],
 [84.6, 56.499999999999

In [ ]:
FalseIdentifyANOTHER = []
for i in range(89):
  FalseIdentifyANOTHER.append([100*thresholding(0.1 + 0.01*i),100*false_identify(0.1 + 0.01*i),])
for i in range(10):
  FalseIdentifyANOTHER.append([100*thresholding(0.99 + 0.001*i),100*false_identify(0.99 + 0.001*i)])

for i in range(10):
  FalseIdentifyANOTHER.append([100*thresholding(0.999 + 0.0001*i),100*false_identify(0.999 + 0.0001*i),])

for i in range(10):
  FalseIdentifyANOTHER.append([100*thresholding(0.9999 + 0.00001*i),100*false_identify(0.9999 + 0.00001*i),])

for i in range(9):
  FalseIdentifyANOTHER.append([100*thresholding(0.99999 + 0.000001*i),100*false_identify(0.99999 + 0.000001*i),])

FalseIdentifyANOTHER.append([100*thresholding(max_value),100*false_identify(max_value),])
FalseIdentifyANOTHER

[[89.7, 10.299999999999999],
 [89.7, 10.299999999999999],
 [89.7, 10.100000000000001],
 [89.5, 10.100000000000001],
 [89.5, 10.0],
 [89.3, 10.0],
 [89.2, 10.0],
 [89.1, 10.0],
 [89.1, 10.0],
 [89.1, 10.0],
 [89.1, 9.9],
 [89.1, 9.9],
 [89.1, 9.8],
 [88.9, 9.8],
 [88.9, 9.8],
 [88.8, 9.8],
 [88.8, 9.8],
 [88.8, 9.8],
 [88.8, 9.8],
 [88.8, 9.8],
 [88.7, 9.700000000000001],
 [88.6, 9.700000000000001],
 [88.6, 9.6],
 [88.6, 9.6],
 [88.6, 9.3],
 [88.6, 9.3],
 [88.5, 9.3],
 [88.4, 9.2],
 [88.3, 9.0],
 [88.1, 8.799999999999999],
 [88.0, 8.799999999999999],
 [87.9, 8.6],
 [87.8, 8.6],
 [87.8, 8.5],
 [87.8, 8.3],
 [87.7, 8.3],
 [87.4, 8.200000000000001],
 [87.4, 8.1],
 [87.4, 8.1],
 [87.0, 7.9],
 [86.8, 7.7],
 [86.8, 7.5],
 [86.7, 7.3999999999999995],
 [86.5, 7.3],
 [85.8, 7.1],
 [85.7, 7.000000000000001],
 [85.39999999999999, 6.800000000000001],
 [85.2, 6.6000000000000005],
 [85.1, 6.1],
 [84.89999999999999, 5.800000000000001],
 [84.89999999999999, 5.800000000000001],
 [84.6, 5.7],
 [84.5, 5.6

# Class-adaptive threshold

# Run #1

In [ ]:
prediction_known = prediction_known_ensemble_1
prediction_unknown = prediction_unknown_ensemble_1

In [ ]:
ThresholdNeverSeenBefores = []
for i in range(10):
  ThresholdNeverSeenBefores.append(np.max(prediction_unknown[:,i]))

In [ ]:
C_count0 = 0
for i in range(100):
  if prediction_known[i,0] > ThresholdNeverSeenBefores[0] and np.argmax(prediction_known[i]) == 0:
    C_count0 += 1
  else:
    C_count0 += 0
C_count0

6

In [ ]:
mistake0 = 0
for i in range(100):
  if np.argmax(prediction_known[i]) != 0 and max(prediction_known[i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[i])]:
    mistake0 += 1
    print(i)
  else:
    mistake0 += 0
mistake0
print("Number of mistakes in this class:",mistake0)

Number of mistakes in this class: 0


In [ ]:
C_count1 = 0
for i in range(100):
  if prediction_known[100 + i,1] > ThresholdNeverSeenBefores[1] and np.argmax(prediction_known[100 + i]) == 1:
    C_count1 += 1
  else:
    C_count1 += 0
C_count1

42

In [ ]:
mistake1 = 0
for i in range(100):
  if np.argmax(prediction_known[100 + i]) != 1 and max(prediction_known[100 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[100 + i])]:
    mistake1 += 1
    print(100 + i)
  else:
    mistake1 += 0
mistake1
print("Number of mistakes in this class:",mistake1)

Number of mistakes in this class: 0


In [ ]:
C_count2 = 0
for i in range(100):
  if prediction_known[200 + i,2] > ThresholdNeverSeenBefores[2] and np.argmax(prediction_known[200 + i]) == 2:
    C_count2 += 1
  else:
    C_count2 += 0
C_count2

15

In [ ]:
mistake2 = 0
for i in range(100):
  if np.argmax(prediction_known[200 + i]) != 2 and max(prediction_known[200 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[200 + i])]:
    mistake2 += 1
    print(200 + i)
  else:
    mistake2 += 0
mistake2
print("Number of mistakes in this class:",mistake2)

Number of mistakes in this class: 0


In [ ]:
C_count3 = 0
for i in range(100):
  if prediction_known[300 + i,3] > ThresholdNeverSeenBefores[3] and np.argmax(prediction_known[300 + i]) == 3:
    C_count3 += 1
  else:
    C_count3 += 0
C_count3

89

In [ ]:
mistake3 = 0
for i in range(100):
  if np.argmax(prediction_known[300 + i]) != 3 and max(prediction_known[300 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[300 + i])]:
    mistake3 += 1
    print(300 + i)
  else:
    mistake3 += 0
mistake3
print("Number of mistakes in this class:",mistake3)

Number of mistakes in this class: 0


In [ ]:
C_count4 = 0
for i in range(100):
  if prediction_known[400 + i,4] > ThresholdNeverSeenBefores[4] and np.argmax(prediction_known[400 + i]) == 4:
    C_count4 += 1
  else:
    C_count4 += 0
C_count4

71

In [ ]:
mistake4 = 0
for i in range(100):
  if np.argmax(prediction_known[400 + i]) != 4 and max(prediction_known[400 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[400 + i])]:
    mistake4 += 1
    print(400 + i)
  else:
    mistake4 += 0
print("Number of mistakes in this class:",mistake4)

403
Number of mistakes in this class: 1


In [ ]:
ThresholdNeverSeenBefores[np.argmax(prediction_known[403])] = max(prediction_known[403])

In [ ]:
mistake4 = 0
for i in range(100):
  if np.argmax(prediction_known[400 + i]) != 4 and max(prediction_known[400 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[400 + i])]:
    mistake4 += 1
    print(400 + i)
  else:
    mistake4 += 0
print("Number of mistakes in this class:",mistake4)

Number of mistakes in this class: 0


In [ ]:
C_count5 = 0
for i in range(100):
  if prediction_known[500 + i,5] > ThresholdNeverSeenBefores[5] and np.argmax(prediction_known[500 + i]) == 5:
    C_count5 += 1
  else:
    C_count5 += 0
C_count5

99

In [ ]:
mistake5 = 0
for i in range(100):
  if np.argmax(prediction_known[500 + i]) != 5 and max(prediction_known[500 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[500 + i])]:
    mistake5 += 1
    print(500 + i)
  else:
    mistake5 += 0
print("Number of mistakes in this class:",mistake5)

Number of mistakes in this class: 0


In [ ]:
C_count6 = 0
for i in range(100):
  if prediction_known[600 + i,6] > ThresholdNeverSeenBefores[6] and np.argmax(prediction_known[600 + i]) == 6:
    C_count6 += 1
  else:
    C_count6 += 0
C_count6

2

In [ ]:
mistake6 = 0
for i in range(100):
  if np.argmax(prediction_known[600 + i]) != 6 and max(prediction_known[600 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[600 + i])]:
    mistake6 += 1
    print(600 + i)
  else:
    mistake6 += 0
print("Number of mistakes in this class:",mistake6)

611
Number of mistakes in this class: 1


In [ ]:
ThresholdNeverSeenBefores[np.argmax(prediction_known[611])] = max(prediction_known[611])

In [ ]:
mistake6 = 0
for i in range(100):
  if np.argmax(prediction_known[600 + i]) != 6 and max(prediction_known[600 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[600 + i])]:
    mistake6 += 1
    print(600 + i)
  else:
    mistake6 += 0
print("Number of mistakes in this class:",mistake6)

Number of mistakes in this class: 0


In [ ]:
C_count7 = 0
for i in range(100):
  if prediction_known[700 + i,7] > ThresholdNeverSeenBefores[7]  and np.argmax(prediction_known[700 + i]) == 7:
    C_count7 += 1
  else:
    C_count7 += 0
C_count7

52

In [ ]:
mistake7 = 0
for i in range(100):
  if np.argmax(prediction_known[700 + i]) != 7 and max(prediction_known[700 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[700 + i])]:
    mistake7 += 1
    print(700 + i)
  else:
    mistake7 += 0
print("Number of mistakes in this class:",mistake7)

Number of mistakes in this class: 0


In [ ]:
C_count8 = 0
for i in range(100):
  if prediction_known[800 + i,8] > ThresholdNeverSeenBefores[8] and np.argmax(prediction_known[800 + i]) == 8:
    C_count8 += 1
  else:
    C_count8 += 0
C_count8

0

In [ ]:
mistake8 = 0
for i in range(100):
  if np.argmax(prediction_known[800 + i]) != 8 and max(prediction_known[800 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[800 + i])]:
    mistake8 += 1
    print(800 + i)
  else:
    mistake8 += 0
print("Number of mistakes in this class:",mistake8)

888
Number of mistakes in this class: 1


In [ ]:
ThresholdNeverSeenBefores[np.argmax(prediction_known[888])] = max(prediction_known[888])

In [ ]:
mistake8 = 0
for i in range(100):
  if np.argmax(prediction_known[800 + i]) != 8 and max(prediction_known[800 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[800 + i])]:
    mistake8 += 1
    print(800 + i)
  else:
    mistake8 += 0
print("Number of mistakes in this class:",mistake8)

Number of mistakes in this class: 0


In [ ]:
C_count9 = 0
for i in range(100):
  if prediction_known[900 + i,9] > ThresholdNeverSeenBefores[9] and np.argmax(prediction_known[900 + i]) == 9:
    C_count9 += 1
  else:
    C_count9 += 0
C_count9

88

In [ ]:
mistake9 = 0
for i in range(100):
  if np.argmax(prediction_known[900 + i]) != 9 and max(prediction_known[900 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[900 + i])]:
    mistake9 += 1
    print(900 + i)
  else:
    mistake9 += 0
print("Number of mistakes in this class:",mistake9)

Number of mistakes in this class: 0


In [ ]:
C_count0 = 0
for i in range(100):
  if prediction_known[i,0] > ThresholdNeverSeenBefores[0] and np.argmax(prediction_known[i]) == 0:
    C_count0 += 1
  else:
    C_count0 += 0

C_count1 = 0
for i in range(100):
  if prediction_known[100 + i,1] > ThresholdNeverSeenBefores[1] and np.argmax(prediction_known[100 + i]) == 1:
    C_count1 += 1
  else:
    C_count1 += 0

C_count2 = 0
for i in range(100):
  if prediction_known[200 + i,2] > ThresholdNeverSeenBefores[2] and np.argmax(prediction_known[200 + i]) == 2:
    C_count2 += 1
  else:
    C_count2 += 0

C_count3 = 0
for i in range(100):
  if prediction_known[300 + i,3] > ThresholdNeverSeenBefores[3] and np.argmax(prediction_known[300 + i]) == 3:
    C_count3 += 1
  else:
    C_count3 += 0

C_count4 = 0
for i in range(100):
  if prediction_known[400 + i,4] > ThresholdNeverSeenBefores[4] and np.argmax(prediction_known[400 + i]) == 4:
    C_count4 += 1
  else:
    C_count4 += 0

C_count5 = 0
for i in range(100):
  if prediction_known[500 + i,5] > ThresholdNeverSeenBefores[5] and np.argmax(prediction_known[500 + i]) == 5:
    C_count5 += 1
  else:
    C_count5 += 0

C_count6 = 0
for i in range(100):
  if prediction_known[600 + i,6] > ThresholdNeverSeenBefores[6] and np.argmax(prediction_known[600 + i]) == 6:
    C_count6 += 1
  else:
    C_count6 += 0

C_count7 = 0
for i in range(100):
  if prediction_known[700 + i,7] > ThresholdNeverSeenBefores[7]  and np.argmax(prediction_known[700 + i]) == 7:
    C_count7 += 1
  else:
    C_count7 += 0

C_count8 = 0
for i in range(100):
  if prediction_known[800 + i,8] > ThresholdNeverSeenBefores[8] and np.argmax(prediction_known[800 + i]) == 8:
    C_count8 += 1
  else:
    C_count8 += 0

C_count9 = 0
for i in range(100):
  if prediction_known[900 + i,9] > ThresholdNeverSeenBefores[9] and np.argmax(prediction_known[900 + i]) == 9:
    C_count9 += 1
  else:
    C_count9 += 0



In [ ]:
FinalDistributionOver10ClassesRun1 = [[1,C_count0],[2,C_count1],[3,C_count2],[4,C_count3],[5,C_count4],[6,C_count5],[7,C_count6],[8,C_count7],[9,C_count8],[10,C_count9]]
FinalDistributionOver10ClassesRun1

[[1, 6],
 [2, 42],
 [3, 15],
 [4, 80],
 [5, 71],
 [6, 99],
 [7, 2],
 [8, 16],
 [9, 0],
 [10, 88]]

In [ ]:
mistake0 = 0
for i in range(100):
  if np.argmax(prediction_known[i]) != 0 and max(prediction_known[i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[i])]:
    mistake0 += 1
    print(i)
  else:
    mistake0 += 0

mistake1 = 0
for i in range(100):
  if np.argmax(prediction_known[100 + i]) != 1 and max(prediction_known[100 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[100 + i])]:
    mistake1 += 1
    print(100 + i)
  else:
    mistake1 += 0

mistake2 = 0
for i in range(100):
  if np.argmax(prediction_known[200 + i]) != 2 and max(prediction_known[200 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[200 + i])]:
    mistake2 += 1
    print(200 + i)
  else:
    mistake2 += 0

mistake3 = 0
for i in range(100):
  if np.argmax(prediction_known[300 + i]) != 3 and max(prediction_known[300 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[300 + i])]:
    mistake3 += 1
    print(300 + i)
  else:
    mistake3 += 0

mistake4 = 0
for i in range(100):
  if np.argmax(prediction_known[400 + i]) != 4 and max(prediction_known[400 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[400 + i])]:
    mistake4 += 1
    print(400 + i)
  else:
    mistake4 += 0

mistake5 = 0
for i in range(100):
  if np.argmax(prediction_known[500 + i]) != 5 and max(prediction_known[500 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[500 + i])]:
    mistake5 += 1
    print(500 + i)
  else:
    mistake5 += 0

mistake6 = 0
for i in range(100):
  if np.argmax(prediction_known[600 + i]) != 6 and max(prediction_known[600 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[600 + i])]:
    mistake6 += 1
    print(600 + i)
  else:
    mistake6 += 0

mistake7 = 0
for i in range(100):
  if np.argmax(prediction_known[700 + i]) != 7 and max(prediction_known[700 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[700 + i])]:
    mistake7 += 1
    print(700 + i)
  else:
    mistake7 += 0

mistake8 = 0
for i in range(100):
  if np.argmax(prediction_known[800 + i]) != 8 and max(prediction_known[800 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[800 + i])]:
    mistake8 += 1
    print(800 + i)
  else:
    mistake8 += 0

mistake9 = 0
for i in range(100):
  if np.argmax(prediction_known[900 + i]) != 9 and max(prediction_known[900 + i]) > ThresholdNeverSeenBefores[np.argmax(prediction_known[900 + i])]:
    mistake9 += 1
    print(900 + i)
  else:
    mistake9 += 0

NumberOfMistakesAfterThrAdj = [mistake0, mistake1, mistake2, mistake3, mistake4, mistake5, mistake6, mistake7, mistake8, mistake9]
NumberOfMistakesAfterThrAdj

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Average over 10 classes

In [ ]:
(C_count0 + C_count1 + C_count2 + C_count3 + C_count4 + C_count5 + C_count6 + C_count7 + C_count8 + C_count9)/1000

0.419

# Average dataset

In [ ]:
import numpy as np

all_datasets = [FinalDistributionOver10ClassesRun1, FinalDistributionOver10ClassesRun2, FinalDistributionOver10ClassesRun3, FinalDistributionOver10ClassesRun4]

new_dataset = []

for x_value in range(1, 11):
    y_values = []

    for dataset in all_datasets:
        for data_point in dataset:
            if data_point[0] == x_value:
                y_values.append(data_point[1])

    avg_y = np.mean(y_values)
    std_y = np.std(y_values)

    new_dataset.append([x_value, avg_y, std_y])

for data_point in new_dataset:
    print(data_point)


[1, 8.0, 1.224744871391589]
[2, 41.0, 2.23606797749979]
[3, 16.25, 2.165063509461097]
[4, 77.5, 2.598076211353316]
[5, 70.5, 3.3541019662496847]
[6, 98.25, 0.82915619758885]
[7, 3.25, 1.299038105676658]
[8, 17.75, 1.7853571071357126]
[9, 0.0, 0.0]
[10, 90.25, 2.48746859276655]


In [ ]:
new_dataset

[[1, 8.0, 1.224744871391589],
 [2, 41.0, 2.23606797749979],
 [3, 16.25, 2.165063509461097],
 [4, 77.5, 2.598076211353316],
 [5, 70.5, 3.3541019662496847],
 [6, 98.25, 0.82915619758885],
 [7, 3.25, 1.299038105676658],
 [8, 17.75, 1.7853571071357126],
 [9, 0.0, 0.0],
 [10, 90.25, 2.48746859276655]]